In [ ]:
from json.decoder import JSONDecodeError
import subprocess
import json
import base64
import socket
import time
 
#--------------Connect---------------
HOST = ""  
PORT = 5555

print("Waiting for connection...")
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((HOST, PORT))
s.listen()
conn, addr = s.accept()
conn.setblocking(0)

print('Connected by', addr)
connected = True
#-------------------------------------

#-------------Run Base Function---------------
CameraStream = subprocess.Popen(['/home/m5stack/payload/bin/camera_stream'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
CameraStream.stdin.write("_{\"stream\":1}\r\n".encode('utf-8'))
CameraStream.stdin.flush()
#---------------------------------------------

img = ""

def CameraStream_CheckImg(doc):
    global img
    if ('img' in doc):
        img = doc['img']
        
def CameraStream_SendUpdate():
    global conn
    conn.sendall(img.encode())
    conn.sendall('\n'.encode())
    


while True:
    try:
        doc = json.loads(CameraStream.stdout.readline().decode('utf-8')) 

        CameraStream_CheckImg(doc)   

        if connected:
            try:
                CameraStream_SendUpdate()
            except Exception as e: 
                if e.errno == 32 or e.errno == 104:
                    print("Connection lost")
                    conn.detach();
                    connected = False
        else:
            print("Waiting for connection...")
            conn, addr = s.accept()
            conn.setblocking(0)
            print('Connected by', addr) 
            connected = True
            
    except Exception as e: 
        print(e)
        pass